<p style="text-align: center;font-size: 40pt">Overview of registration algorithms</p>

# Overview 

Objectives of this lesson:

- introduce point cloud registration
- show the research work that lead to modern registration algorithms
- decompose modern registration algorithms as a list of high-level steps

Hidden custom latex commands here $ \curvearrowright$

----
[comment]: <> (General commands)
$\DeclareMathOperator*{\argmin}{arg\,min}$
$\DeclareMathOperator{\error}{error}$
$\DeclareMathOperator*{\match}{match}$
$\DeclareMathOperator{\distance}{d}$
$\DeclareMathOperator{\outlier}{outlier}$
$\DeclareMathOperator{\weight}{w}$
$\DeclareMathOperator{\datafilter}{datafilter}$
$\newcommand{\mat}[1]{\mathbf{#1}}$
$\newcommand{\point}[2][]{{}^{#1}\mathbf{#2}}$
$\newcommand{\frame}[1]{\mathcal{#1}}$
$\newcommand{\shape}[2][]{{}^{#1}\mathcal{#2}}$
$\newcommand{\matches}[1]{\mathcal{#1}}$
$\newcommand{\transformation}[3][T]{{}_{#2}^{#3}\mat{#1}}$
$\newcommand{\weights}[1]{\mathcal{#1}}$
$\newcommand{\textcomma}{\quad,}$
$\newcommand{\textdot}{\quad.}$
$\newcommand{\bmat}[1]{\begin{bmatrix}#1\end{bmatrix}}$
----

# Introduction
The scope of this module is to present *registration* algorithms and their use in mobile robotics.
Registration algorithms associate sets of coordinates into a common coordinate system by minimizing the alignment error.
This allows to integrate coordinates from different sources into a larger model.

Although they can be quite an abstract and technical concept, registration solutions already had an impact on the artistic field and popular culture.
Photographers proficiently use image registration to build photograph composites achieving different looks-and-feels.
The [Brenizer method](https://en.wikipedia.org/wiki/Brenizer_Method) is an exemplary technique that is applied to achieve dramatic depth of field using panoramic image stitching ([Figure 1.1](#regInPhotography) - Top).
Another example is High Dynamic Range (HDR) photographs, where multiple images at different exposure levels need to be precisely overlaid to retrieve details in shaded and highlighted areas ([Figure 1.1](#regInPhotography) - Bottom).
Nowadays, even the latest cellphones have the capacity to build panoramic images from a series of pictures taken based on a visual guidelines that direct the user to move the camera viewfinder at the optimal position for the next picture.
As for the specific case of 3D mapping application, cinematographers are depicting possible uses of registration algorithms in several recent science fiction movies.
For instance, in the remake of *Total Recall* (Colombia Pictures, 2012), an armed intervention team employed an array of hundreds of tiny cameras in a dangerous room leading to a 3D reconstruction of the area used to monitor potential threats within couple of seconds.
If you have Netflix, go to the time 25:40 up 27:00 to to see the scene, it's interestingly accurate.
Another closely related potential application was the used by a geologist of [flying drones carrying laser rangefinders](https://www.youtube.com/watch?v=yO-eduvo904) to explore an alien facility in *Prometheus* (Twentieth Century Fox, 2012).

<p id="regInPhotography" style="text-align: center;">
    <img src="images/mosaic.png" width="32.25%"/>
    <img src="images/wooden_horse_rectilinear.png" width="42%"/>
    <img src="images/sanFranciscoHDR.jpg" width="56.25%"/> <br/>
    <b>Figure 1.1:</b>  Example of image registrations used in photography.
    <em>Top</em>: Brenizer method using the open source software Hugin to stitch multiple images.
    <em>Bottom</em> : HDR composite of the San Francisco harbor using the open source software Luminance HDR to overlay three images.
</p>

More at the research level, current applications include: robotic exploration in harsh environments, organ reconstruction to improve medical diagnostics and object reconstruction for reverse engineering. 
Although registration using 2D images can be part of the same group of solutions, we focus on systems where depth information is already available (e.g., from lidars) and is mainly used for resolving misalignment error.
We refer to the latter type as *geometric registration*.
However, some parallels with image registrations will be made throughout this module when relevant.

A simplified example of geometric registration is illustrated in [Figure 1.2](#exampleReg).
A scene with a large tree, a lamppost and a bench was scanned using a laser rangefinder from two different poses.
As laser points are indistinguishable, only their location information is available to resolve the alignment error.
In that example, the point cloud in light green and with the horizontal ground is used as our fixed reference coordinates.
[Figure 1.2](#exampleReg)-*Left* shows the starting position of the two scans.
The overlaid point cloud in dark blue has a misalignment error shifting it to the left with a tilt angle.
This initial misalignment is represented with dark red lines in [Figure 1.2](#exampleReg)-*Middle*.
Although all individual points are similar, their proximity to other points gives enough information to automatically align the two point clouds ([Figure 1.2](#exampleReg)-*Right*).
<p id="exampleReg" style="text-align: center;">
    <img src="images/icp_example_side_1.png" width="32.375%"/>
    <img src="images/icp_example_side_2.png" width="32.375%"/>
    <img src="images/icp_example_side_5.png" width="32.375%"/> <br/>
    <b>Figure 1.2:</b> Examples of geometric registration between a reference point cloud (light green points) and a reading point cloud (dark blue points).
    <em>Left</em>: Initial position of the two point clouds.
    <em>Middle</em> : Alignment error (dark red lines).
    <em>Right</em> : Final alignment of the two point clouds.
</p>

# Early Solutions
As an interesting historical note, in an early publication by [Hurley and Cattell [1962]](https://onlinelibrary.wiley.com/doi/abs/10.1002/bs.3830070216), registration is presented as an *Orthogonal Procrustes* problem.
The name Procrustes refering to a bandit from the Greek mythology who made his victims fit on his bed by either stretching their limbs or cutting them off.
Theseus eventually defeated Procrustes using the same violent procedure ([Figure 1.3](#procrustes)).
Nowadays, the reference to the *Orthogonal Procrustes* problem is not often used in the scientific literature, but it illustrates well the idea.
<p id="procrustes" style="text-align: center;">
    <img src="images/Procrustes.jpg" width="35.25%"/> <br/>
    <b>Figure 1.3:</b> Theseus <em>adjusting</em> Procrustes to the size of his own bed. Photograph provided by Marie-Lan Nguyen / Wikimedia Commons.
</p>

While working more specifically on 3D-shape primitives, [Faugeras and Hebert [1986]](https://journals.sagepub.com/doi/abs/10.1177/027836498600500302) defined closed-form distances to minimize point-to-point and plane-to-plane alignment error.
The proposed method solved translation and rotation as a two-step procedure.
Later, a solution proposed by [Walker et al. [1991]](https://www.sciencedirect.com/science/article/abs/pii/104996609190036O) resolved together rotation and translation error using dual quaternions.
The registration problem concretizes itself further in a survey of geometric matching algorithms and geometric representations for point sets, curves, surfaces, volumes, and their respective space-time trajectories [[Besl, 1988]](https://ieeexplore.ieee.org/document/5966).
At this time, the main experimental validation was using Computer-aided design (CAD) models with simple shapes.
The first mention of the name Iterative Closest Point (ICP) was proposed by [Besl and McKay [1992]](https://ieeexplore.ieee.org/document/121791).
They expressed the problem as follows:

> “Given 3-D data in a sensor coordinate system, which describes a data shape that may correspond to a model shape, and given a model shape in a model coordinate system in a different geometric shape representation, estimate the optimal rotation and translation that aligns, or registers, the model shape and the data shape minimizing the distance between the shapes and thereby allowing determination of the equivalence of the shapes via a mean-square distance metric.”

Yes, that is an awfully long sentence.
If we rephrase that, it means that we can register two point clouds using a rigid transformation and minimizing a mean-square distance.

In their work, the proof of the solution convergence is demonstrated under the assumption that the number of associated points, or their weight, remains constant.
Unless two identical shapes are registered together, outliers that are not present in both shapes need to be identified.
This problems is observed by [Champleboux et al. [1992]](https://ieeexplore.ieee.org/document/223223), while developing early registration solutions for medical applications.
They report failures when wrong initial transformations are used in combination with scans having low overlap ratio.
During the same years, [Chen and Medioni [1991]](https://ieeexplore.ieee.org/document/132043) work with dense laser scans of statues and, shortly later, scans of tooth mockups [[Chen and Medioni, 1992]](https://www.sciencedirect.com/science/article/abs/pii/026288569290066C).
They propose a registration solution based on the minimization of point-to-plane alignment errors, which is still quite often used nowadays.

Even though a large volume of theoretical works was published on advanced geometric primitives (e.g., planes, curves, quadrics), [Zhang [1994]](https://link.springer.com/article/10.1007%2FBF01427149) states that primitives derived from points are too sensitive to noise and are not stable in moving systems with current (1994) sensing capabilities. 
Thus, he concludes that points were more reliable.
[Zhang [1994]](https://link.springer.com/article/10.1007%2FBF01427149) pioneers the idea of using ICP-based solutions for outdoor robotic applications.
He proposes a generic framework for symmetric match, but considers only one direction of registration as an approximation to save computation costs. 
He highly emphasizes the importance of removing spurious pairs and gives the first characterization of fast subsampling solutions.
In addition, he highlighted the fact that outlier rejection is required for robotic applications, and that the proof of ICP convergence stated by [Besl and McKay [1992]](https://ieeexplore.ieee.org/document/121791) cannot hold for most of the robotics applications. 
In the outlook section of his work, he already mentions the use of uncertainty on the initial alignment, based on Kalman filters and Mahalanobis distance, and the need to handle dynamic elements.

# Division and Explosion of the Field
Within only two years, four main application types already emerged from the possibilities to register 3D point clouds: object reconstructions [[Chen and Medioni, 1991]](https://ieeexplore.ieee.org/document/132043), non-contact inspections [[Besl and McKay, 1992]](https://ieeexplore.ieee.org/document/121791), medical and surgery support [[Champleboux et al., 1992]](https://ieeexplore.ieee.org/document/223223) and autonomous vehicle navigation [[Zhang, 1993]](https://link.springer.com/chapter/10.1007/3-540-57233-3_60).
Here is a video showing an example of a medical application with ICP being used for [teeth scanning](https://www.youtube.com/watch?v=X5CvlUZ5DpQ).
Publications in specialized journals for computer vision, robotics and medical imaging slowly divided the types of *interesting* problems to be solved.
We can still read in current literature that the credits for being the first article to provide a solution differ from authors in different fields.

The field of registration crystalized with its first survey on medical image registration covering the years 1993 to 1998 [[Maintz and Viergever, 1998]](https://www.sciencedirect.com/science/article/abs/pii/S1361841501800268).
It took 12 years for a specialized survey of 3D registration in computer vision to appear [[Bowyer et al., 2006]](https://www.sciencedirect.com/science/article/abs/pii/S1077314205000822). 
This module is based on the first large scale review adapted for Robotics application[[Pomerleau et al., 2015]](https://hal.archives-ouvertes.fr/hal-01178661/document).

ICP is a popular algorithm due to its simplicity: its general idea is easy to understand and to implement.
However, the basic algorithm only works well in ideal cases.
This led to hundreds of variations around the original algorithm that were published on numerous different experimental scenarios (see [Figure 1.4](#nbPublications)).
This highlights both the usefulness of ICP and the difficulty to find a single versatile version.
<p id="nbPublications" style="text-align: center;">
    <img src="images/nbPublications.png" width="60%"/> <br/>
    <b>Figure 1.4:</b> Evolution of the number of publications over the years.
    Results were obtained for the keywords <em>Iterative Closest Point</em> appearing in the abstract or the title of publications.
    The dark blue area is computed based on IEEE Xplore database and the light green area from the Scopus database.
</p>

In [Figure 1.4](#nbPublications), one can observe an increasing number of publications appearing around the year 2000.
In robotics, this coincides with the advent of a 2D solution for pose estimations demonstrated with a SICK rangefinder [[Lu and Milios, 1997]](https://link.springer.com/article/10.1023/A:1008854305733) and of the basis of Simultaneous Localization and Mapping (SLAM) algorithms [[Thrun et al., 1998]](https://link.springer.com/article/10.1023/A:1007436523611).
Prior to the arrival of the [SICK LMS-200](https://www.sick.com/ca/en/detection-and-ranging-solutions/2d-lidar-sensors/lms2xx/lms200-30106/p/p109843) in robotics [[Pfister et al., 2002]](https://ieeexplore.ieee.org/document/1014782), most of the sensors used were custom-made.
This situation renders experiments difficult to replicate by other researchers.
In those years, 2D lasers appeared as a viable solution for navigation over sonars, which were traditionally used [[Thrun et al., 1998]](https://link.springer.com/article/10.1023/A:1007436523611).
The 3D real-time applications were then not accessible due to high computation costs leading to an increased research focus toward 2D solutions for autonomous navigation, while medical and computer graphics fields continued there research in 3D.
At the same time in computer vision, the seminal work of [Rusinkiewicz and Levoy [2001]](https://ieeexplore.ieee.org/document/924423) on ICP algorithm comparisons led to significant progress in the scan registration field.
The experiments employed simulated 3D scans, highlighting different spatial constraints and sensor noises.
Results mainly focused on the rapidity of convergence and the final precision of different solutions helping to select more appropriate solutions in further applications.

With the arrival of more standard sensors, researchers in robotics pushed research for 2D registration algorithms, in order to deal with larger environments with faster platforms [[Bosse et al., 2004]](https://journals.sagepub.com/doi/10.1177/0278364904049393), but 3D slowly came back [[Nüchter et al., 2004]](https://ieeexplore.ieee.org/document/1308117).
Since no comparison framework exists, the selection of an appropriate variant for particular experimental conditions is difficult.
This is a major challenge because registration is at the front-end of the mapping pipeline, and the arbitrary nature of its selection affects the results of all subsequent steps of more advanced robotic tasks.
Even the early work of [Eggert et al. [1998]](https://www.sciencedirect.com/science/article/abs/pii/S1077314298906671) highlights the difficulty to compare with other solutions given the lack of metric over common data sets.
In their survey, [Maintz and Viergever [1998]](https://www.sciencedirect.com/science/article/abs/pii/S1361841501800268) point the fact that proper accuracy studies are just starting; the problem being that the results provided are too specific. 
In addition, they highlight the imprecise use of the terms accuracy, precision and robustness. 
They suggest to set up public databases and validation protocols, but foresee logistic, costs and efforts as incoming problem to those solutions.

Recently, the demand for a stronger experimental methodology in robotics was also stressed by [Amigoni et al. [2009]](https://link.springer.com/article/10.1007/s10514-009-9137-8).
The authors survey different SLAM publications to highlight proper evaluation metrics that are applied to SLAM algorithms.
Three principles of an experimental methodology in science (i.e, comparison, reproducibility/repeatability and justification/explanation) are translated in requirements for stronger SLAM results.
As stated in their paper, a sound methodology should allow researchers to gain an insight about *intrinsic* (e.g., computational time, parameters used, parameter behaviors) and *extrinsic* (e.g., accuracy, precision) quantities.
The authors report that, even though comparisons between algorithms are present in SLAM publications, very few researchers can reuse the same protocol and directly compare their results without having to re-implement other solutions.

Our survey of publications on ICP stopped around 2013.
With the introduction of the [Microsoft Kinect in 2010](https://en.wikipedia.org/wiki/Kinect), another wave of publications was expected, similar to what was observed following the widespread utilization of SICK rangefinders.
The Kinect is a handheld camera sensor connected via USB to a computer that produces both depth and color readings.
Such RGB-D sensors augment accessibility to object modeling and indoor mapping research [[Henry et al., 2012]](https://journals.sagepub.com/doi/abs/10.1177/0278364911434148).
This also opens the door to a mix of hybrid algorithms using features and descriptors without the need of expertise in sensor calibration.
RGB-D cameras have different characteristics than laser-based sensors, such as a higher density of points at a higher frequency, but covering a more restricted Field of View (FoV).
A smaller FoV means less time to compute the registration before the sensor trajectory reduce the overlap to an unusable range.
Having access to a higher frame rate with an optimized ICP solution shows that hand-waved sensor trajectory was trackable with real-time constraints [[Pomerleau et al.,
2011]](https://ieeexplore.ieee.org/abstract/document/6094861).
The Velodyne HDL-64E, first commercialized for the DARPA Urban Challenge in 2007, optimized its FoV to cover the expected trajectory of a ground vehicle.
To cope with the high speed of a car, the sensor delivered a high data rate at 1.3 M points per second, bringning the real-time constraint to another level.
Those two sensors were the latest publication catalysts for the field of registration in mobile robotics, field often modulated by the development of new hardwares.

# Algorithm Overview
The goal of geometric registration is to be able to represent a shape, called <tt>reading</tt>, in the same coordinate frame as another, called <tt>reference</tt>.
This is equivalent to finding the transformation of <tt>reading</tt> that best aligns it to <tt>reference</tt>.
We will define a shape $\shape{S}$ as a set of points including both geometric and non-geometric information.
Geometric information is affected by a spatial transformation; this part of the dimension of a point will be called a *feature*.
Features are typically coordinates of points, surface normals or tangent vectors.
Non-geometric information is not affected by spatial transformation; this part of the dimension of a point will be called a *descriptor*.
Example of descriptors are color, temperature, and identifiers to name few.
The main steps of the algorithm are shown as a chart in [Figure 1.5](#systemGeneralGraph).
The specific quantities exchanged between boxes will be explained in the following paragraphs.

<p id="systemGeneralGraph" style="text-align: center;">
    <img src="images/regFramework.jpg" width="100%"/> <br/>
    <b>Figure 1.5:</b> Generic scheme used for registration algorithms.
    Blue arrows represent features, while red arrows represent descriptors.
</p>

Most algorithms actually apply some filters on the shapes in order to help the registration.
There are mainly two uses of such filters.
The first one is to remove some points that do not bring any valuable information for the registration.
As the complexity of the algorithm is linear in the number of points, reducing this number can have a significant impact on the time of registration.
The second use of filters can be to add information to the point.
The typical example is the inference of local structural properties of the shape, such as surface normal vectors or curvatures.
This information, which is usually not present in the raw sensor data, can allow for better registration through a more precise association of the points, or the computation of the error to minimize.

More formally, let $\shape[A]{P}$ be the shape representing <tt>reading</tt> in a frame of reference $\frame{A}$ and $\shape[B]{Q}$ the shape representing <tt>reference</tt> in its frame of reference $\frame{B}$.
The aim of registration is to estimate the transformation $\transformation{A}{B}$ by minimizing an error function $\error(\shape{P'}, \shape{Q})$:


<div style="text-align: right">
    <a id="generic_minimization"></a>
\begin{aligned}
\transformation[\hat{T}]{A}{B} = 
\argmin_{\transformation{}{}}\left(\error\left(\transformation{}{}\left(\shape[A]{P}\right), \shape[B]{Q}\right)\right)
    \textcomma
\end{aligned}
    (1.1)
</div>

    
where $\transformation{}{}(\shape{S})$ is the application of the geometric transformation with parameters $\transformation{}{}$ to the shape $\shape{S}$.

One specificity of geometric registration is that the error function is computed on pairs of points that have been associated between the two shapes.
The classical association is done by finding the closest point in <tt>reference</tt> of each point in <tt>reading</tt>.
Ideally the association should be between points that, when the two shapes are aligned, are the closest in position.
This problem is called *data association*, *point matching*, or *correspondence finding* depending on the literature.
Association solving can be done purely on the features but can also be improved by using descriptors.

Formally, let $\matches{M}=\match(\shape{P}, \shape{Q})=\left\{(\point{p}, \point{q}): \point{p}\in\shape{P}, \point{q}\in\shape{Q}\right\}$ be the set of matches between $\shape{P}$ and $\shape{Q}$.
We will come back to the detail of the function $\match(\cdot)$ later in the [lesson Point cloud association](7.2-lesson_association.ipynb), but for now just imagine that $\matches{M}$ is a long list of tuples containing a point from $\shape{P}$ and a point from $\shape{Q}$.
Knowing that each points are paired together, the error function simplify to the form:

$$
    \error(\shape{P}, \shape{Q}) = \sum_{(\point{p}, \point{q})\in\matches{M}}{\distance(\point{p}, \point{q})}
    \textcomma
$$

where $\distance(\cdot)$ is a function defining a distance between two points.
You already know the Euclidean distance, but there are others too.
In order to make this error function more robust, *outliers* are sometimes identified and removed from the list of matches.
In addition, weights $\weights{W}=\outlier(\matches{M})=\left\{\weight(\point{p}, \point{q}): \forall (\point{p}, \point{q})\in\matches{M}\right\}$ can be associated to the matches so as to increase or decrease their influence in the error function:

$$
    \error(\shape{P}, \shape{Q}) = \sum_{(\point{p}, \point{q})\in\matches{M}}{\weight(\point{p}, \point{q}) \distance(\point{p}, \point{q})}
    \textcomma
$$

where $\weight(\cdot)$ is a weighting function that will be explained in the [lesson Outliers](7.3-lesson_outliers.ipynb).
It is clear that minimizing this error function with an ideal association yields the best estimate for $\transformation{A}{B}$ ([Equation 1.1](#generic_minimization)).
However, unless the descriptors are discriminative enough, which is typically the case with visual descriptors, the association can generally not be perfectly solved.
The idea of ICP is that even with an imperfect association, minimizing the error yields a better estimates that, in turn, allows for better association.
Concretely, the idea is to build a sequence of transformations $\transformation{i-1}{i}$ that are successively applied to $\shape{P}$ and minimize iteratively the alignment error.
At a given iteration, a set of matches $\matches{M}_i$ is computed from the given relative position of the points.
Then, based on those matches, a new transformation $\transformation{i}{i+1}$ is computed by minimizing the error:

<div style="text-align: right">
    <a id="error_minimization"></a>
\begin{aligned}
\transformation{i}{i+1} \gets \argmin\limits_{\transformation{}{}}\left(\error\left(\transformation{}{}\left(\shape[i]{P'}\right), \mathcal{Q}'\right)\right)
    \textdot
\end{aligned}
    (1.2)
</div>

We will see in the [lesson Error minimization](7.4-lesson_error_minimization.ipynb) few examples on how to compute this equation.
Finally, the estimate of the transformation between the two original shapes is the composition of all intermediary transformations:

<div style="text-align: right">
<a id="final_transform"></a>
$$
    \transformation[\hat{T}]{A}{B} = \transformation{i}{B} \cdots \transformation{1}{2} \; \transformation{0}{1} \; \transformation{A}{0}_{init}
    \textcomma
$$
    (1.3)
</div>

where $i$ is the current number of iterations, and $\transformation{A}{0}_{init}$ an initial transformation when $i=0$.
The generic procedure is summarized in [Algorithm 1](#icp).


<p id="icp" style="margin-left: 27%;">
    <hr style="margin-left: 27%;margin-right: 29.5%;"/>
    <span style="margin-left: 28%;"> <b>Algorithm 1</b> Summary of ICP algorithm.</span>
    <hr style="margin-left: 27%;margin-right: 29.5%;"/>
    <p style="margin-left: 28%;">
        <b>Require:</b> $\shape[A]{P}$ $\hspace{150pt}$ &#8618;<tt>reading</tt> <br/>
        <b>Require:</b> $\shape[B]{Q}$ $\hspace{150pt}$ &#8618;<tt>reference</tt> <br/>
        <b>Require:</b> $\transformation{}{}_{init}$ $\hspace{146pt}$ &#8618;initial transformation <br/>
        <p style="margin-left: 30%;">
            $\shape[A]{P'} \gets \datafilter(\shape[A]{P})$ $\hspace{100pt}$ &#8618;data filters <br/>
            $\shape[B]{Q'} \gets \datafilter(\shape[B]{Q})$ $\hspace{100pt}$ &#8618;data filters <br/>
            $\transformation{i-1}{i} \gets \transformation{}{}_{init}$ <br/>
            <b>repeat</b> <br/>
            <p style="margin-left: 32%;">
                $\shape[i]{P'} \gets \transformation{i-1}{i}(\shape[i-1]{P'})$ $\hspace{97pt}$ &#8618;move <tt>reading</tt> <br/>
                $\matches{M}_i \gets \match(\shape[i]{P'}, \shape{Q'})$ $\hspace{85pt}$ &#8618;associate points <br/>
                $\weights{W}_i \gets \outlier(\matches{M}_i)$ $\hspace{98pt}$ &#8618;filter outliers <br/>
                $\transformation{i}{i+1} \gets \argmin\limits_{\transformation{}{}}\left(\error\left(\shape[i]{P'}, \shape{Q'}\right)\right)$
            </p>
        </p>
        <p style="margin-left: 30%;">
            <b>until</b> convergence
        </p>
    </p>
    <p style="margin-left: 28%;">
        <b>Ensure:</b> $\transformation[\hat{T}]{A}{B} = \transformation{i}{B} \cdots \transformation{1}{2} \; \transformation{0}{1} \; \transformation{A}{0}_{init}$
    </p>
    <hr style="margin-left: 27%;margin-right: 29.5%;"/>
</p>

</br>

# Conclusion
You should do the following activities to enhance your understanding of the concepts viewed in this lesson:
- modify the markdown by adding your own notes using `> my notes`; and
- complete the tables [Symbol definitions](#Symbol-definitions) and [Glossary](#Glossary) and add your own definitions.

Next lessons (parallel):
- [Data association](7.2-lesson_association.ipynb)
- [Processing point clouds](7.2-lesson_processing.ipynb)

## Symbol definitions

| &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Symbol &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; | Definition                    |
|:--------------------------------:|:-----------------------------------------------------:|
| $\transformation[\hat{T}]{A}{B}$ | Estimate of the transformation between frames A and B |
| $\error(\shape{P}, \shape{Q})$   |  Alignment error between $\shape{P}$ and $\shape{Q}$  |
| $\match(\shape{P}, \shape{Q}) $  | Set of matches between $\shape{P}$ and $\shape{Q}$    |
| $\outlier(\matches{M})$          | Set of weights associated to the matches $\matches{M}$ |
| $\transformation{}{}(\shape{S})$ | Application of the transformation $\transformation{}{}$ to $\shape{S}$ |
| $\distance(\point{p}, \point{q})$| Distance between points $\point{p}$ and $\point{q}$   |
| $\weight(\point{p}, \point{q})$  | Weight of the match between points $\point{p}$ and $\point{q}$  |

## Glossary

| English   | Français   | Definition |
|-----------|------------|------------|
|           |            |            |